In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import random
import numpy as np
import time

In [2]:
def load_data(filename): # Cargar JSON
    data = None
    with open(filename,"r") as in_file:
        data = json.load(in_file)
    return data

In [8]:
datos = load_data('testfile.json')
PM = json_normalize(datos['PM'])
VM = json_normalize(datos['VM'])

In [46]:
random.seed(123)
np.random.seed(123)
ValorError = 1000000000 # ¿Cómo hacer infinito?
consumoMEJOR = ValorError # ¿Cómo hacer infinito?
size_VM = len(VM) # Nº de máquinas virtuales
size_PM = len(PM) # Nº de máquinas físicas
contador = 0.0
deseado = 8000
L_asignaciones = [] # Lista de asignaciones
L_asignacionesNEW = []
asig = [[0],[0],[0],[0]]    # Inicializar asignaciones (Padre, Madre, Hijo1, Hijo2)
consumo = [[0],[0],[0],[0]] # Inicializar consumos     (Padre, Madre, Hijo1, Hijo2)

iteracion = 0
# Fórmula del consumo de una máquina
def formula(Wmax, Wmin, UsoCPU):
    Wtotal = Wmin + (UsoCPU) * (Wmax - Wmin) 
    return Wtotal
# Mirar si es posible una asignación y devolver consumo total
def consumo_asignacion(h):
    PM_Estado = [[0]*size_PM, [0]*size_PM] # Estado de una máquina física ['usocpu', 'usomem']
    asignacion_es_posible = True # Variable para ver si es posible la asignación
    i_VM = 0 # Index de la posición de la lista de asignación 'h', que equivale a la máquina virtual
    for i_PM in h:
        print(i_PM)
        # Mirar si el estado de la máquina física permite asignar la máquina virtual
        if(100 - PM_Estado[0][i_PM]) >= ((VM.loc[i_VM,'cpuusage']+PM.loc[i_PM,'vmwareoverhead']) / PM.loc[i_PM,'cores']) and ((PM.loc[i_PM,'mainmemory'] - PM_Estado[1][i_PM]) >= VM.loc[i_VM,'mainmemory']):
            print(i_PM)
            # Actualizar estado de la cpu de la máquina física
            PM_Estado[0][i_PM] += (VM.loc[i_VM,'cpuusage'] / PM.loc[i_PM,'cores'])
            # Actualizar estado de la mem de la máquina física
            PM_Estado[1][i_PM] += VM.loc[i_VM,'mainmemory']
            # Apuntar a la siguiente posición de la lista
            i_VM += 1
        else:
            # La asignación no es posible
            asignacion_es_posible = False
            # Salir del 'for'
            break
    if asignacion_es_posible:
        total = 0
        for i in range(0,size_PM):
            if PM_Estado[0][i] != 0: # Mirar si la máquina 'i' tiene uso_cpu != 0
                actual = formula(PM.loc[i,'powermax'],PM.loc[i,'powermin'],PM_Estado[0][i]+(PM.loc[i,'vmwareoverhead']/PM.loc[i,'cores']))
                total += actual
        return total
    else:
        return ValorError # Un consumo W infinito
generacion = 0
start_time = time.time() # Guardar tiempo de comienzo de ejecución
while True:
    if contador >= deseado: # Si durante X recorridos el consumoMEJOR no ha cambiado, hemos encontrado el deseado
        break
    finalizador = random.randint(20, 40) # Buscar entre 20 y 40 nuevas asignaciones
    while True:
        asignacion = np.random.choice(size_PM, size_VM) # Crear una lista de tamaño 'size_VM' llenando
                                                        # con números de 0 a 'size_PM'
        # Ejemplo: en la posición '0' hay un '1' -> la máquina virtual '0' se ha asignado a la máquina física '1'
        consumoNEW = consumo_asignacion(asignacion) # Obtener consumo de asignación posible o no
        generacion += 1
        if consumoNEW != ValorError: # Mirar si el consumo es diferente de Error
            # Añadir a lista de asignaciones
            L_asignaciones.append(asignacion)
            # Restar al finalizador
            finalizador -= 1
            if consumoMEJOR > consumoNEW:
                asignacionMEJOR = asignacion
                consumoMEJOR = consumoNEW
                end_time = time.time()
                print("Nuevo consumo energético: "+str(consumoMEJOR)+"W, Generaciones: "+str(generacion)+", Población: "+str(len(L_asignaciones))+", Tiempo de ejecución: "+str((end_time - start_time))+"s")
                contador = 0
        contador += 1
        if finalizador == 0:
            break
    random.shuffle(L_asignaciones)
    
    for permutaciones in range(0,1):
        for l in range(0,len(L_asignaciones),2):
            asig[0] = L_asignaciones[l]
            asig[1] = L_asignaciones[(l+1)%len(L_asignaciones)]
            k = random.randint(1, size_VM - 1)
            asig[2] = np.concatenate((asig[0][:k],asig[1][k:]),axis=0) 
            asig[3] = np.concatenate((asig[1][:k],asig[0][k:]),axis=0)

            consumo[0] = consumo_asignacion(asig[0]) # Consumo Padre
            consumo[1] = consumo_asignacion(asig[1]) # Consumo Madre
            consumo[2] = consumo_asignacion(asig[2]) # Consumo Hijo1
            consumo[3] = consumo_asignacion(asig[3]) # Consumo Hijo2
            i = consumo.index(min(consumo)) # Obtener índice del consumo menor
            consumoNEW = consumo[i]
            asignacion = asig[i]
            if consumoNEW!=ValorError:
                L_asignacionesNEW.append(asignacion) # Coger la asignación con el menor consumo
                if consumoMEJOR > consumoNEW:
                    asignacionMEJOR = asignacion
                    consumoMEJOR = consumoNEW
                    end_time = time.time()
                    print("Nuevo consumo energético: "+str(consumoMEJOR)+"W, Generaciones: "+str(generacion)+", Población: "+str(len(L_asignaciones))+", Tiempo de ejecución: "+str((end_time - start_time))+"s")
                    contador = 0
                contador += 1
        L_asignaciones = L_asignacionesNEW
        random.shuffle(L_asignaciones)
        L_asignacionesNEW = [] # Reiniciar lista de asignaciones nuevas
        iteracion += 1
# FIN
end_time = time.time()
print("\nMejor consumo energético: "+str(consumoMEJOR)+"W")
print("\nMejor asignación:")
i_VM = 0
for i_PM in asignacionMEJOR:
    print("VM: "+str(i_VM)+"|PM: "+str(i_PM))
    i_VM += 1

print("Tiempo de ejecución final: "+str((end_time - start_time))+"s")

Nuevo consumo energético: 8014.5W, Generaciones: 1, Población: 1, Tiempo de ejecución: 0.004739522933959961s
Nuevo consumo energético: 7474.0W, Generaciones: 2, Población: 2, Tiempo de ejecución: 0.009288311004638672s
Nuevo consumo energético: 6940.0W, Generaciones: 5, Población: 5, Tiempo de ejecución: 0.023631572723388672s
Nuevo consumo energético: 6787.5W, Generaciones: 6, Población: 6, Tiempo de ejecución: 0.027995586395263672s
Nuevo consumo energético: 6449.5W, Generaciones: 11, Población: 11, Tiempo de ejecución: 0.049712181091308594s
Nuevo consumo energético: 6211.5W, Generaciones: 15, Población: 15, Tiempo de ejecución: 0.06697583198547363s
Nuevo consumo energético: 5613.0W, Generaciones: 63, Población: 33, Tiempo de ejecución: 0.8246901035308838s
Nuevo consumo energético: 5353.0W, Generaciones: 193, Población: 65, Tiempo de ejecución: 3.4920132160186768s
Nuevo consumo energético: 5318.5W, Generaciones: 387, Población: 56, Tiempo de ejecución: 7.712159872055054s
Nuevo consumo e